# MLP

In [1]:
import os
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, ops
from tqdm import tqdm
import wandb

In [2]:
root_dir = 'COVID-19_Radiography_Dataset'
train_dir = os.path.join(root_dir, 'lbp', 'train')
val_dir = os.path.join(root_dir, 'lbp', 'val')

In [ ]:
wandb.login(key=os.getenv('WANDB_KEY'))
wandb.init(project='mlp_local_binary_patterns', name='LBP default - no aug')

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Transforms

The script shown below calculates the mean and standard deviation for the training set. These values are then used to apply the standarization to the whole dataset.

In [ ]:
dataset = datasets.ImageFolder(train_dir, transform=Grayscale())
loader = DataLoader(dataset, 64, shuffle=True)

# Initialize variables for mean and std calculation
mean = 0.0
std = 0.0
nb_samples = 0

for data, _ in loader:
    data = data.to(device)
    batch_samples = data.size(0)  # number of images in the batch
    data = data.view(batch_samples, -1)  # flatten the channel and spatial dimensions
    mean += data.mean(1).sum(0)
    std += data.std(1).sum(0)
    nb_samples += batch_samples

mean /= nb_samples
std /= nb_samples

print("Mean:", mean)
print("Std:", std)

These transforms are applied for standarization and data augmentation purposes

In [4]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Grayscale(num_output_channels=1),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
        # transforms.RandomResizedCrop(size=299, scale=(0.8, 1.0)),
        transforms.ToTensor(),
        # transforms.Normalize(mean=0.0197, std=0.0083)  # Normalize the image (uniform LBP)
        transforms.Normalize(mean=0.6133, std=0.3372)  # Normalize the image (default LBP)
        # transforms.Normalize(mean=0.2283, std=0.3103)  # Normalize the image (ror LBP)
        # transforms.Normalize(mean=0.0772, std=0.1784)  # Normalize the image (var LBP)
    ]),
    'val': transforms.Compose([
        transforms.ToTensor(),
        transforms.Grayscale(),
        # transforms.Normalize(mean=0.0197, std=0.0083)  # Normalize the (uniform LBP)
        transforms.Normalize(mean=0.6133, std=0.3372)  # Normalize the image (default LBP)
        # transforms.Normalize(mean=0.2283, std=0.3103)  # Normalize the image (ror LBP)
        # transforms.Normalize(mean=0.0772, std=0.1784)  # Normalize the image (var LBP)
    ]),
}

## Model

### MLP PyTorch module

In [5]:
class MLP(nn.Module):
  def __init__(self, input_dim, output_dim, hidden_layers=[128], activation='relu', dropout=0):
    super(MLP, self).__init__()

    if activation == "relu":
      activation_func = lambda: nn.ReLU()
    elif activation == "prelu":
      activation_func = lambda: nn.PReLU()
    # You can add other activation functions here
    else:
      raise NotImplementedError("Activation function not supported")

    self.flatten = nn.Flatten()
    self.input_layer = nn.Linear(input_dim, hidden_layers[0])
    self.input_activation = activation_func()

    self.hidden_layers = nn.ModuleList([])
    for index in range(len(hidden_layers)):
        next = index + 1
        if next < len(hidden_layers):
          self.hidden_layers.append(nn.Linear(hidden_layers[index], hidden_layers[next]))
          self.hidden_layers.append(nn.BatchNorm1d(hidden_layers[next]))
        else:
          self.hidden_layers.append(nn.Linear(hidden_layers[index], hidden_layers[index]))
          self.hidden_layers.append(nn.BatchNorm1d(hidden_layers[index]))
        self.hidden_layers.append(activation_func())
        if dropout > 0:
          self.hidden_layers.append(nn.Dropout(dropout))

    self.output_layer = nn.Linear(hidden_layers[-1], output_dim)


  def forward(self, x):
    x = self.flatten(x)
    x = self.input_layer(x)
    x = self.input_activation(x)

    for layer in self.hidden_layers:
      x = layer(x)

    x = self.output_layer(x)
    return x

### Definitions and initializations

In [6]:
batch_size = 64
input_features = 299 * 299
hidden_layers = [256, 256]
num_classes = 4
dropout = 0.5
activation = 'prelu'

In [7]:
# Load datasets
train_dataset = datasets.ImageFolder(train_dir, transform=data_transforms['train'])
val_dataset = datasets.ImageFolder(val_dir, transform=data_transforms['val'])

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size, shuffle=False)

In [11]:
# Model definition
model = MLP(input_dim=input_features,
            hidden_layers=hidden_layers,
            output_dim=num_classes,
            dropout=dropout,
            activation=activation).to(device)

# Define loss function and optimizer (replace with your choices)
criterion = nn.CrossEntropyLoss()  # Assuming classification task
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [12]:
# Training loop
num_epochs = 20
best_val_loss = float('inf')
epochs_no_improve = 0
patience = 3
model_save_path = 'COVID-19_Radiography_Dataset/models/best_model_lbp.pth'

for epoch in range(num_epochs):
  running_loss = 0.0
  correct = 0
  total = 0
  model.train()
  
  train_progress = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training", unit="batch")
  for inputs, labels in train_progress:
    inputs, labels = inputs.to(device), labels.to(device)

    optimizer.zero_grad()

    outputs = model(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item() * inputs.size(0)
    _, predicted = torch.max(outputs, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

    train_progress.set_postfix({"Loss": running_loss / total, "Acc": correct / total})

  epoch_loss = running_loss / len(train_loader.dataset)
  epoch_acc = correct / total

  print(f'Epoch {epoch+1}/{num_epochs}')
  print(f'Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

  # wandb.log({"epoch": epoch + 1, "train_loss": epoch_loss, "train_accuracy": epoch_acc})

  model.eval()
  val_loss = 0.0
  val_correct = 0
  val_total = 0

  # Validation phase with progress bar
  val_progress = tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Validation", unit="batch")
  with torch.no_grad():
    for inputs, labels in val_progress:
      inputs, labels = inputs.to(device), labels.to(device)

      outputs = model(inputs)
      loss = criterion(outputs, labels)

      val_loss += loss.item() * inputs.size(0)
      _, predicted = torch.max(outputs, 1)
      val_total += labels.size(0)
      val_correct += (predicted == labels).sum().item()

      val_progress.set_postfix({"Loss": val_loss / val_total, "Acc": val_correct / val_total})

  val_loss = val_loss / len(val_loader.dataset)
  val_acc = val_correct / val_total

  print(f'Val Loss: {val_loss:.4f} Acc: {val_acc:.4f}')

  # wandb.log({"epoch": epoch + 1, "val_loss": val_loss, "val_accuracy": val_acc})

  # Check for improvement
  if val_loss < best_val_loss:
    best_val_loss = val_loss
    epochs_no_improve = 0
    torch.save(model.state_dict(), model_save_path)  # Save the best model
  else:
    epochs_no_improve += 1

  if epochs_no_improve >= patience:
    print("Early stopping triggered!")
    break


print("Training complete!")

Epoch 1/20 - Training: 100%|██████████| 265/265 [02:03<00:00,  2.14batch/s, Loss=0.959, Acc=0.597]


Epoch 1/20
Train Loss: 0.9593 Acc: 0.5966


Epoch 1/20 - Validation: 100%|██████████| 67/67 [00:19<00:00,  3.42batch/s, Loss=0.743, Acc=0.704]


Val Loss: 0.7428 Acc: 0.7041


Epoch 2/20 - Training: 100%|██████████| 265/265 [02:01<00:00,  2.18batch/s, Loss=0.798, Acc=0.672]


...

Val Loss: 0.4998 Acc: 0.8179


Epoch 19/20 - Training: 100%|██████████| 265/265 [01:53<00:00,  2.33batch/s, Loss=0.391, Acc=0.855]


Epoch 19/20
Train Loss: 0.3914 Acc: 0.8548


Epoch 19/20 - Validation: 100%|██████████| 67/67 [00:17<00:00,  3.85batch/s, Loss=0.529, Acc=0.807]

Val Loss: 0.5288 Acc: 0.8066
Early stopping triggered!
Training complete!
